In [2]:

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import arange
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from xgboost import XGBRegressor

from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [3]:
df_train = pd.read_csv("./dataset/train.csv")
df_train.shape

(1460, 81)

In [4]:
df_test = pd.read_csv("./dataset/test.csv")
df_test.shape

(1459, 80)

### variable numérique

In [5]:
df_train['LotArea'].isna().sum()
df_train['LotArea'].describe()
#df['LotArea'].unique()

count      1460.000000
mean      10516.828082
std        9981.264932
min        1300.000000
25%        7553.500000
50%        9478.500000
75%       11601.500000
max      215245.000000
Name: LotArea, dtype: float64

In [7]:
df_train.loc[df_train['TotalBsmtSF'] == 0, 'TotalBsmtSF'] = df_train['TotalBsmtSF'].quantile(0.5)
#df['TotalBsmtSF'].describe()

In [8]:
#df[['GrLivArea']].fillna((df['GrLivArea'].mean()), inplace=False)
df_train_num = df_train[['GrLivArea', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'LotArea']] 

df_train_num.head()

,GrLivArea,GarageArea,TotalBsmtSF,1stFlrSF,LotArea
0,1710,548,856.0,856,8450
1,1262,460,1262.0,1262,9600
2,1786,608,920.0,920,11250
3,1717,642,756.0,961,9550
4,2198,836,1145.0,1145,14260


In [9]:
scaler = MinMaxScaler()
df_train_num_scaled = scaler.fit_transform(df_train_num)

scaler = StandardScaler().fit(df_train_num_scaled)
df_train_num_scaled = scaler.transform(df_train_num_scaled)

In [10]:
df_train_num = pd.DataFrame(df_train_num_scaled, columns = df_train_num.columns.tolist())

### Variable cétégorielle

In [11]:
df_train_cat = df_train[['OverallQual', 'GarageCars', 'FullBath', 'TotRmsAbvGrd']].astype(str)
df_train_cat.describe()

,OverallQual,GarageCars,FullBath,TotRmsAbvGrd
count,1460,1460,1460,1460
unique,10,5,4,12
top,5,2,2,6
freq,397,824,768,402


In [13]:
df_test_cat = df_test[['OverallQual', 'GarageCars', 'FullBath', 'TotRmsAbvGrd']].astype(str)
df_test_cat.describe()

,OverallQual,GarageCars,FullBath,TotRmsAbvGrd
count,1459,1459,1459,1459
unique,10,7,5,12
top,5,2.0,2,6
freq,428,770,762,442


In [ ]:
Il faut concaténer les datasets train et test pour avoir toutes les "dummies"

In [17]:
df_train_test_cat = pd.concat([df_train_cat, df_test_cat])
df_train_test_cat.shape

(2919, 4)

In [18]:
df_train_test_cat = pd.get_dummies(df_train_test_cat)
df_train_test_cat.head()

,OverallQual_1,OverallQual_10,OverallQual_2,OverallQual_3,OverallQual_4,OverallQual_5,OverallQual_6,OverallQual_7,OverallQual_8,OverallQual_9,...,TotRmsAbvGrd_14,TotRmsAbvGrd_15,TotRmsAbvGrd_2,TotRmsAbvGrd_3,TotRmsAbvGrd_4,TotRmsAbvGrd_5,TotRmsAbvGrd_6,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
df_new = df_train_num.join(df_train_test_cat.head(1460))
df_new.shape

(1460, 46)

In [44]:
df_new.columns

Index(['GrLivArea', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'LotArea',
       'OverallQual_1', 'OverallQual_10', 'OverallQual_2', 'OverallQual_3',
       'OverallQual_4', 'OverallQual_5', 'OverallQual_6', 'OverallQual_7',
       'OverallQual_8', 'OverallQual_9', 'GarageCars_0', 'GarageCars_0.0',
       'GarageCars_1', 'GarageCars_1.0', 'GarageCars_2', 'GarageCars_2.0',
       'GarageCars_3', 'GarageCars_3.0', 'GarageCars_4', 'GarageCars_4.0',
       'GarageCars_5.0', 'GarageCars_nan', 'FullBath_0', 'FullBath_1',
       'FullBath_2', 'FullBath_3', 'FullBath_4', 'TotRmsAbvGrd_10',
       'TotRmsAbvGrd_11', 'TotRmsAbvGrd_12', 'TotRmsAbvGrd_13',
       'TotRmsAbvGrd_14', 'TotRmsAbvGrd_15', 'TotRmsAbvGrd_2',
       'TotRmsAbvGrd_3', 'TotRmsAbvGrd_4', 'TotRmsAbvGrd_5', 'TotRmsAbvGrd_6',
       'TotRmsAbvGrd_7', 'TotRmsAbvGrd_8', 'TotRmsAbvGrd_9'],
      dtype='object')

In [21]:
# Variables explicatives
X = df_new

# Variable cible
y = df_train['SalePrice']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.22, random_state=42)

'''
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train) # X_train_scaled
X_test = scaler.transform(X_test) # X_test_scaled

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
'''

'\nscaler = MinMaxScaler()\n\nX_train = scaler.fit_transform(X_train) # X_train_scaled\nX_test = scaler.transform(X_test) # X_test_scaled\n\nscaler = StandardScaler().fit(X_train)\n\nX_train = scaler.transform(X_train)\nX_test = scaler.transform(X_test)\n'

### Linear

In [23]:
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)

y_pred_test = lr.predict(X_test)

In [24]:
mse_train = mean_squared_error(y_train, y_pred_train)

mse_test = mean_squared_error(y_test, y_pred_test)

print("MSE train lr:", mse_train)
print("MSE test lr:", mse_test)

MSE train lr: 1220800218.6335232
MSE test lr: 1270015797.577586


In [26]:
mae_train = mean_absolute_error(y_train, y_pred_train)

mae_test = mean_absolute_error(y_test, y_pred_test)

print("MAE train lr:", mae_train)
print("MAE test lr:", mae_test)

mean_price = df_train['SalePrice'].mean()

print("\nRelative error", mae_test / mean_price)

MAE train lr: 22733.227529330066
MAE test lr: 22770.952426802043

Relative error 0.1258611646619617


### Ridge

In [27]:
ridge_reg = RidgeCV(alphas= (0.001, 0.01, 0.1, 0.3, 0.7, 1, 10, 50, 100))
ridge_reg.fit(X_train, y_train) 

RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 3.e-01, 7.e-01, 1.e+00, 1.e+01, 5.e+01,
       1.e+02]))

In [28]:
print('alpha sélectionné par c-v :', ridge_reg.alpha_)
print('score train :', ridge_reg.score(X_train, y_train))
print('score test :', ridge_reg.score(X_test, y_test))

alpha sélectionné par c-v : 1.0
score train : 0.7975412013880926
score test : 0.8203666015574294


In [29]:
ridge_pred_train = ridge_reg.predict(X_train)

ridge_pred_test = ridge_reg.predict(X_test)

In [30]:
mse_train = mean_squared_error(ridge_pred_train, y_train)

mae_test = mean_squared_error(ridge_pred_test, y_test)

print('mse train :', mse_train)
print('mse test :', mae_test)

mse train : 1225834592.9741454
mse test : 1290613605.223902


In [31]:
mae_train = mean_absolute_error(ridge_pred_train, y_train)

mae_test = mean_absolute_error(ridge_pred_test, y_test)

print("MAE train lr:", mae_train)
print("MAE test lr:", mae_test)

mean_price = df_train['SalePrice'].mean()

print("\nRelative error", mae_test / mean_price)

MAE train lr: 22749.810314200393
MAE test lr: 22914.231491844123

Relative error 0.1266531065034741


### ElasticNet

In [32]:
model_en = ElasticNetCV(cv=8, l1_ratio=(0.1, 0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.99), 
                        alphas=(0.001, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 0.8, 1.0), max_iter=50000)

model_en.fit(X_train, y_train)

ElasticNetCV(alphas=(0.001, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 0.8, 1.0),
             cv=8, l1_ratio=(0.1, 0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.99),
             max_iter=50000)

In [33]:
pred_train = model_en.predict(X_train)
pred_test = model_en.predict(X_test)


In [34]:
mse_train = mean_squared_error(pred_train, y_train)

mae_test = mean_squared_error(pred_test, y_test)

print('mse train :', mse_train)
print('mse test :', mae_test)

mse train : 1235040547.119731
mse test : 1318278098.3573215


In [35]:
mae_train = mean_absolute_error(pred_train, y_train)

mae_test = mean_absolute_error(pred_test, y_test)

print("MAE train lr:", mae_train)
print("MAE test lr:", mae_test)

mean_price = df_train['SalePrice'].mean()

print("\nRelative error", mae_test / mean_price)

MAE train lr: 22756.280064692455
MAE test lr: 23145.415507324356

Relative error 0.12793092259540548


### XGBoost

In [36]:
model_xgbr = XGBRegressor(learning_rate=0.09, max_depth=4, n_estimators=500, tree_method='auto', subsample=0.64)

In [37]:
model_xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.09, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.64,
             tree_method='auto', validate_parameters=1, verbosity=None)

In [38]:
y_pred_train = model_xgbr.predict(X_train)

y_pred_test = model_xgbr.predict(X_test)

In [39]:
mse_train = mean_squared_error(y_train, y_pred_train)

mse_test = mean_squared_error(y_test, y_pred_test)

print("MSE train lr:", mse_train)
print("MSE test lr:", mse_test)

MSE train lr: 51821241.039492026
MSE test lr: 895172439.8189751


In [41]:
mae_train = mean_absolute_error(y_train, y_pred_train)

mae_test = mean_absolute_error(y_test, y_pred_test)

print("MAE train lr:", mae_train)
print("MAE test lr:", mae_test)

mean_price = df_train['SalePrice'].mean()

print("\nRelative error", mae_test / mean_price)

MAE train lr: 5583.872109786907
MAE test lr: 20294.98180318323

Relative error 0.1121758105969876


#### save a model
#### https://www.geeksforgeeks.org/saving-a-machine-learning-model/

In [42]:
import joblib

model_file = "./model/model_xgbr.pkl"
joblib.dump(model_xgbr, model_file)

['./model/model_xgbr.pkl']